In [2]:
def generateCol(n):
    names = []
    for i in range(n):
        col_name = ''
        while i >= 0:
            col_name = chr(i % 26 + 65) + col_name
            i = i // 26 - 1
        names.append(col_name)
    return names


In [29]:
import os
import pandas as pd

# Paths and Initialization
folderPath = "FOLDER"
masterPath = "Master datasheet.xlsx"
outputPath = "master\\master.xlsx"

# Read the master template and initialize column names
master = pd.read_excel(masterPath)
masterColnames = master.columns  # Preserve original column names
n = len(master.columns)
Colnames = generateCol(n)
master.columns = Colnames  # Temporary column names for internal processing

# Loop through all files in the folder
for foldername in os.listdir(folderPath):
    paths = folderPath+"\\"+foldername
    for filename in os.listdir(paths):
    # for filename in os.listdir(folderPath):
        try:
            # Read column mapping sheet
            Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
            if not {'From', 'To'}.issubset(Col_directions.columns):
                raise ValueError(f"Mapping sheet {filename} must contain 'From' and 'To' columns.")

            # Read the current file in the folder
            filePath = os.path.join(paths, filename)
            fileRead = pd.read_excel(filePath)
            temp_data = []  # Temporary storage for processed rows
            
            # Breaking the measurement columns into M1, M2, M3
            if 'Measurement' in fileRead.columns:
                fileRead[['m1', 'm2', 'm3']] = fileRead['Measurement'].str.split(' x ', expand=True)
                fileRead[['m1', 'm2', 'm3']] = fileRead[['m1', 'm2', 'm3']].apply(pd.to_numeric, errors='coerce')
          
            # Assign dynamic column names to the current file
            n = len(fileRead.columns)
            Colnames = generateCol(n)
            fileRead.columns = Colnames

            # Process rows and map data based on column directions
            for _, row in fileRead.iterrows():
                if pd.isna(row['A']):  # Stop processing if empty row
                    break
                
                new_row = {}  # Create a dictionary for the new row
                for _, movement_row in Col_directions.iterrows():
                    if movement_row['From'] in row.index and movement_row['To'] in master.columns:
                        new_row['A'] = filename
                        new_row[movement_row['To']] = row[movement_row['From']]
                temp_data.append(new_row)  # Add the processed row to temporary storage

            # Convert temp_data to a DataFrame and append to master
            temp_df = pd.DataFrame(temp_data)
            master = pd.concat([master, temp_df], ignore_index=True)

        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            continue
        
def calculate_R(row):
    if row['Q'] > 0:
        return ((100 - row['Q']) / 100) * row['P']
    else:
        return ((100 + row['Q']) / 100) * row['P']
    
def calculate_S(row):
    if pd.notna(row['R']) and pd.notna(row['C']):  # Check if both R and C are not NaN
        return round(row['R'] * row['C'])  # Perform calculation and round to 2 decimal places
    return None

master['R'] = master.apply(calculate_R, axis=1)
master['S'] =master.apply(calculate_S,axis=1)
master['I'] = master['I'].apply(lambda x: None if pd.isna(x) or str(x).strip() == '' else x)

# Restore original column names
master.columns = masterColnames

# Save the appended master file
master.to_excel(outputPath, index=False)
print(f"Master file updated and saved at {outputPath}.")


C:\Users\91965\AppData\Local\Temp\ipykernel_9780\2074103261.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  master = pd.concat([master, temp_df], ignore_index=True)


Master file updated and saved at master\master.xlsx.
